In [1]:
import os
import pandas as pd
from textwrap import dedent
import os
import errno

In [ ]:
1944416 2199837 3133323 4000644 5137597 6866109 7824961 8587911 9909020 0560884

In [2]:
def mkdir_p(path):
    """Python version mkdir -p

    Parameters
    ----------
    path : string
           Path to directory to create
    """
    if path:
        try:
            os.makedirs(path)
        except OSError as exc:  # Python >2.5
            if exc.errno == errno.EEXIST and os.path.isdir(path):
                pass
            else:
                raise
def symlink_force(target, link_name):
    try:
        os.symlink(target, link_name)
    except OSError as e:
        if e.errno == errno.EEXIST:
            os.remove(link_name)
            os.symlink(target, link_name)
        else:
            raise e

In [3]:
assembly_common_name_map = {'mm10': 'mouse', 'hg38': 'human'}

sheet_name = 'hg38'
excel_df = pd.read_excel('../data/datasets_info.xlsx', sheet_name=sheet_name)
excel_df

,GEO,SRA,RIBO sample,RNA sample,read lengths,offsets,RIBO adapter,RNA adapter,RIBO strandedness,RNA strandedness,AUC(RiboCop/RiboCode/RiboTaper/ORFscore),metagene_coherence_ribocop
0,GSE35469,SRP010679,SRX118286,SRX118285,"28,29,30","12,13,13",CTGTAGGCAC,CTGTAGGCAC,forward,forward,NaN,NaN
1,NaN,SRP029589,SRX345309,SRX345311,"29,30,32","12,12,13",CTGTAGGCACCATCAAT,CTGTAGGCACCATCAAT,forward,forward,NaN,NaN
2,GSE73136,SRP063852,SRX1254413,"SRX426378 (SRP028887, GSE49831)","28,29,30","12,12,12",None,None,forward,forward,NaN,NaN
3,GSE94454,SRP098789,SRX2536421,SRX2536426,"28,30","12,13",CTGTAGGCACCATCAAT,CTGTAGGCACCATCAAT,forward,forward,NaN,NaN
4,GSE96716,SRP102021,SRX2647167,SRX2647164,"28,29,30,31","12,12,12,12",TCGTATGCCGTCTTCTGCTTG,None,forward,forward,NaN,NaN
5,GSE87328,SRP090415,SRX2189157,SRX2189154,"26,27,28,29,30","12,12,12,12,12",CTGTAGGCACCATCAAT,AGATCGGAAGAGCACAC,forward,reverse,NaN,NaN
6,GSE89183,SRP092068,SRX2268404,SRX2268396,"26,27,28,29,30","12,12,12,12,12",AGATCGGAAGAGCACACGTCT,None,forward,reverse,NaN,NaN
7,GSE96714,SRP102020,SRX2647162,SRX2647160,"29,30","12,12",TCGTATGCCGTCTTCTGCTTG,None,forward,no,NaN,NaN


In [8]:
input_dir_root = '/home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/'
genome_name = 'hg38'
genome_base_path = '/home/cmb-panasas2/skchoudh/genomes/hg38/rpbp_index_mrna'
gtf = '/home/cmb-panasas2/skchoudh/genomes/hg38/annotation/gencode.v25.annotation.gtf'
genome_fasta = '/home/cmb-panasas2/skchoudh/genomes/hg38/fasta/hg38.fa'
ribosomal_index = '/home/cmb-panasas2/skchoudh/genomes/hg38/rpbp_index_rrna'
star_index = '/home/cmb-panasas2/skchoudh/genomes/hg38/rpbp_star_index'


In [10]:
for index, row in excel_df.iterrows():
    lengths = list(row['read lengths'].strip().split(','))
    offsets = row['offsets'].strip().split(',')
    srp = row['SRA']
    ribo_sample = row['RIBO sample']
    rna_sample = row['RIBO sample']
    input_dir = os.path.join(input_dir_root, srp+'_human')
    riboseq_data = os.path.join(input_dir, 'rpbp_results')

    symlink_dir = os.path.join(riboseq_data, 'without-rrna-mapping')
    mkdir_p(symlink_dir)
    ribo_file = os.path.join(input_dir, 'RIBO_CCDS_lengths.bam')
    rna_file = os.path.join(input_dir, 'RNA_CCDS_lengths.bam')
    
    ribo_file_symlink = os.path.join(symlink_dir, 'RIBO_CCDS_lengths.alignments-only.bam')
    rna_file_symlink = os.path.join(symlink_dir, 'RNA_CCDS_lengths.alignments-only.bam')
    
    symlink_force(rna_file, rna_file_symlink)
    print(rna_file, rna_file_symlink)
    symlink_force(ribo_file, ribo_file_symlink)

    hg38_config = """
    genome_name: {}
    genome_base_path: {}
    gtf: {}
    fasta: {}
    ribosomal_index: {}
    star_index: {}

    #adapter_file: /path/to/rp-bp/riboseq-adapters.fa

    riboseq_data: {}

    riboseq_samples:
        RIBO_CCDS_lengths: 
        RNA_CCDS_lengths:


    riboseq_biological_replicates:
        # these must still be the same names from riboseq_samples
        RIBO_CCDS_lengths:
            - RIBO_CCDS_lengths
        RNA_CCDS_lengths:
            - RNA_CCDS_lengths

    note: alignments-only

    orf_note: atg-only

    #models_base: /path/to/rp-bp/models
    max_uncalled: 1
    pre_trim_left: 0
    adapter_sequence: null
    quality_format: sanger


    sjdb_overhang: 50
    align_intron_minc: 20
    align_intron_maxc: 100000
    out_filter_intron_motifsc: RemoveNoncanonicalUnannotated
    out_filter_mismatch_n_maxc: 1
    out_filter_mismatch_n_over_l_maxc: 0.04
    out_filter_typec: BySJout
    out_sam_attributesc: [AS, NH, HI, nM, MD]

    # METAGENE PERIODICITY OPTIONS, OPTIONAL

    # the number of bases upstream of the translation initiation site to begin 
    # constructing the metagene profile. default: 50
    metagene_profile_start_upstream: 50 

    # the number of bases downstream of the translation initiation site to end 
    # the metagene profile. default: 20
    metagene_profile_start_downstream: 20

    # the number of bases upstream of the translation termination site to begin 
    # constructing the metagene profile. default: 50
    metagene_profile_end_upstream: 50

    # the number of bases downstream of the translation termination site to end 
    # the metagene profile. default: 20
    metagene_profile_end_downstream: 20

    # the position, relative to the translation initiation site, to begin 
    # calculating periodicity Bayes factors (inclusive)
    periodic_offset_start: -20

    # the position, relative to the translation initiation site, to stop 
    # calculating periodicity Bayes factors (inclusive)
    periodic_offset_end: 0

    # the length of the profile to use in the models. 

    # N.B. metagene_profile_length + periodic_offset_end must be consistent with 
    # the length of the extracted metagene profile
    metagene_profile_length: 21

    # the number of iterations to use for each chain in the MCMC sampling. The 
    # first half of the iterations are discarded as burn-in samples. All of 
    # the remaining samples are used to estimate the posterior distributions. That 
    # is, we do not use thinning. default: 500

    # N.B. Increasing the number of iterations will increase the running time of
    # the estimation roughly linearly.
    metagene_profile_iterations: 500

    # SHARED MARKOV CHAIN MONTE CARLO OPTIONS, OPTIONAL

    # These affect the MCMC both for estimating metagene profile periodicity and 
    # ORF translation Bayes factors.

    # the random seed for the MCMC sampling. default: 8675309
    seed: 8675309 

    # the number of chains to use in the MCMC sampling. default: 2

    # N.B. Increases in the number of chains should improve the quality of
    # estimation (especially reducing variance), but will increase the running
    # time proportionally. For example, changing the number of chains from 2 to
    # 4 will roughly double the running time of estimation.
    chains: 2

    # PERIODICITY AND OFFSET OPTIONS, OPTIONAL

    # FIXED LENGTHS AND OFFSETS
    # By default, the Rp-Bp pipelines use the periodicity estimation technique to
    # select periodic read lengths and P-site offsets. However, fixed lengths and
    # offsets can be specified with the following options.

    # if this variable is present in the config file with any value, then the 
    # estimated periodic read lengths and offsets will not be used. Instead, 
    # fixed values given by lengths and offsets (below) will be used.

    # uncomment this line to use the fixed lengths and offsets
    use_fixed_lengths: yes

    # a list of read lengths which will be used for creating the profiles if the 
    # use_fixed_lengths option is given. Presumably, these are lengths that have 
    # periodic metagene profiles.

    # This value is ignored if 'use_fixed_lengths' is not set.
    lengths: {}

    # the P-site offset to use for each read length specifed by --lengths if the 
    # use_fixed_lengths option is given. The number of offsets must match the 
    # number of lengths, and they are assumed to match. For example lengths of 
    # [26, 29] with offsets [9, 12] means only reads of lengths 26 bp and 29 bp 
    # will be used to create the profiles. The 26 bp reads will be shifted by 
    # 9 bp in the 5' direction, while reads of length 29 bp will be shifted by 12 bp.

    # This value is ignored if 'use_fixed_lengths' is not set.
    offsets: {}

    # if fixed lengths are not used, the minimum number of reads for a particular 
    # length in the filtered genome profile. Read lengths with fewer than this 
    # number of reads will not be used. default: 1000
    #min_metagene_profile_count: 1000

    # N.B. This value is set artificially low for the example to work!
    min_metagene_profile_count: 10

    # if fixed lengths are not used...
    # if max_metagene_profile_bayes_factor_var is not None, then this is taken as a 
    # hard threshold on the estimated Bayes factor mean. 

    # If min_metagene_bf_likelihood is given, then this is taken as 
    # the boundary value; that is, a profile is "periodic" if:
    #
    #       [P(bf > min_metagene_bf_mean)] > min_metagene_bf_likelihood
    #
    # If both max_metagene_bf_var and min_metagene_bf_likelihood are None, then this 
    # is taken as a hard threshold on the mean for selecting periodic read lengths.
    #
    # If both max_metagene_bf_var and min_metagene_bf_likelihood are given, then both 
    # filters will be applied and the result will be the intersection.
    min_metagene_bf_mean: 5

    # if given, then this is taken as a hard threshold on the estimated Bayes factor 
    # variance. default: None, i.e., this filter is not used. (null in yaml) 
    max_metagene_bf_var: null

    # if given, then this is taken a threshold on the likelihood of periodicity 
    # (see min_metagene_bf_mean description for more details). default: 0.5
    min_metagene_bf_likelihood: 0.5

    # SMOOTHING OPTIONS, OPTIONAL

    # the fraction of the profile to use for smoothing within LOWESS. default: 0.2
    smoothing_fraction: 0.2

    # the number of reweighting iterations to use within LOWESS. Please see the
    # statsmodels documentation for a detailed description of this parameter.
    # default: 0
    smoothing_reweighting_iterations: 0

    # if this value is greater than 0, then ORFs whose length (in nucleotides) 
    # is less than this value will not be smoothed, and neither the Bayes factor 
    # estimates nor the chi-square p-value will be calcualted. default: 0
    min_orf_length: 0

    # if this value is greater than 0, then ORFs whose length (in nucleotides) 
    # is greater than this value will not be smoothed, and neither the Bayes factor 
    # estimates nor the chi-square p-value will be calcualted. default: 0
    max_orf_length: 0

    # ORFs for which the number of in-frame reads is less than this value will not
    # be smoothed, and neither the Bayes factor estimates nor the chi-square
    # p-value will be calculated. default: 5
    min_signal: 5


    # BAYES FACTOR ESTIMATION OPTIONS, OPTIONAL

    # if this flag is in the config file with any value, then only the Rp-chi 
    # pipeline will be performed; namely, the translation models will not be fit 
    # to the data, and the posterior distributions will not be estimated.

    # uncomment this line to only run the Rp-chi pipeline
    #chi_square_only: yes

    # the number of iterations to use for each chain in the MCMC sampling. The first 
    # half of the iterations are discarded as burn-in samples. All of the remaining 
    # samples are used to estimate the posterior distributions. (That is, we do not 
    # use thinning.) default: 200

    # N.B. Increasing the number of iterations will increase the running time of
    # the estimation roughly linearly.
    translation_iterations: 200

    # SELECTION PREDICTED ORFS OPTIONS, OPTIONAL

    # the minimum value for the estimated Bayes factor mean to "predict" that an 
    # ORF is translated. This value is used in conjunction with both min_bf_mean
    # and min_bf_likelihood.
    #
    # If max_bf_var is a positive value, then this is taken as a hard threshold on 
    # the estimated Bayes factor mean. ORFs must meet both the min_bf_mean and
    # max_bf_var filters to be selected as "translated."
    #
    # If min_bf_likelihood is given, then this is taken as the boundary value; that
    # is, an ORF is selected as "translated" if:
    #
    #   [P(bf > min_bf_mean)] > min_bf_likelihood
    #
    # If both max_bf_var and min_bf_likelihood are None, then this is taken as a
    # hard threshold on the mean for selecting translated ORFs.
    #
    # If both max_bf_var and min_bf_likelihood are given, then both filters will 
    # be applied and the result will be the intersection.
    #
    # default: 5
    min_bf_mean: 5

    # the maximum value value for the estimated Bayes factor variance to "predict" 
    # that an ORF is translated. ORFs must meet both the min_bf_mean and max_bf_var 
    # filters to be predicted. See the description for min_bf_mean for more 
    # details. default: null (i.e., this filter is not used by default)
    max_bf_var: null

    # the minimum probability of the BF exceeding min_bf_mean to select an ORF as
    # translated. See the description for min_bf_mean for more details. default: 0.5
    min_bf_likelihood: 0.5

    # for the chi-square test, this value is first Bonferroni corrected 
    # based on the number of ORFs which pass the smoothing filters. It is 
    # then used as the significance threshold to select translated ORFs. default: 0.01
    chisq_significance_level: 0.01
    """.format(genome_name, genome_base_path, gtf,
               genome_fasta, ribosomal_index, star_index,
               riboseq_data, lengths, offsets)
    hg38_config = hg38_config.replace("'","")
    with open('/home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/rpbp/config_{}_human.yaml'.format(srp), 'w') as fh:
        fh.write(dedent(hg38_config))
    to_write = 'run-all-rpbp-instances config_{}_human.yaml --overwrite --num-cpus 16 --logging-level INFO --log-file log.alignments-only_{}.txt --keep-intermediate-files'.format(srp, srp)
    with open('/home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/rpbp/run_rpbp_{}_human.sh'.format(srp), 'w') as fh:
        fh.write(to_write)

/home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/SRP010679_human/RNA_CCDS_lengths.bam /home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/SRP010679_human/rpbp_results/without-rrna-mapping/RNA_CCDS_lengths.alignments-only.bam
/home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/SRP029589_human/RNA_CCDS_lengths.bam /home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/SRP029589_human/rpbp_results/without-rrna-mapping/RNA_CCDS_lengths.alignments-only.bam
/home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/SRP063852_human/RNA_CCDS_lengths.bam /home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/SRP063852_human/rpbp_results/without-rrna-mapping/RNA_CCDS_lengths.alignments-only.bam
/home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/SRP098789_human/RNA_CCDS_lengths.bam /home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/SRP098789_human/rpbp_results/without-rrna-mapping/RNA_CCDS_lengths.alignments-only.bam
/home/cmb-panasas2/skchoudh/benchmark/gene_level/hg38/SRP102021_human/RN

In [ ]:
ribo_file_symlink

In [ ]:
ribo_file

# Mouse

In [11]:
assembly_common_name_map = {'mm10': 'mouse', 'hg38': 'human'}

sheet_name = 'mm10'
excel_df = pd.read_excel('../data/datasets_info.xlsx', sheet_name=sheet_name)
excel_df

,GEO,SRA,RIBO sample,RNA sample,read lengths,offsets,RIBO adapter,RNA adapter,RIBO strandedness,RNA strandedness
0,GSE22001,SRP003554,SRX026871,SRX026872,"28,29,30","12,12,12",TCGTATG,TCGTATG,forward,forward
1,GSE72064,SRP062407,SRX1149649,SRX1149668,"28,29,30,31","12,12,12,12",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
2,GSE84112,SRP078005,SRX1900396,SRX1900402,"26,27,28,29,30","12,12,12,12,12",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
3,GSE89011,SRP091889,SRX2255510,SRX2255511,"26,27,28,29,30","12,12,12,12,12",AGATCGGAAGAGCACACGTCT,AGATCGGAAGAGCACACGTCT,forward,forward
4,GSE102890,SRP115915,SRX3110803,SRX3110807,"29,30,31,32,33,34","12,12,12,13,13,13",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
5,GSE99787,SRP108787,SRX2893684,SRX2893692,"25,26,27,28","12,10,11,12",None,None,forward,reverse


In [12]:
input_dir_root = '/home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/'
genome_name = 'mm10'
genome_base_path = '/home/cmb-panasas2/skchoudh/genomes/mm10/rpbp_index_mrna'
gtf = '/home/cmb-panasas2/skchoudh/genomes/mm10/annotation/gencode.vM11.annotation.gtf'
genome_fasta = '/home/cmb-panasas2/skchoudh/genomes/mm10/fasta/mm10.fa'
ribosomal_index = '/home/cmb-panasas2/skchoudh/genomes/mm10/rpbp_index_rrna'
star_index = '/home/cmb-panasas2/skchoudh/genomes/mm10/rpbp_star_index'


In [14]:
for index, row in excel_df.iterrows():
    lengths = list(row['read lengths'].strip().split(','))
    offsets = row['offsets'].strip().split(',')
    srp = row['SRA']
    ribo_sample = row['RIBO sample']
    rna_sample = row['RIBO sample']
    input_dir = os.path.join(input_dir_root, srp+'_mouse')
    riboseq_data = os.path.join(input_dir, 'rpbp_results')

    symlink_dir = os.path.join(riboseq_data, 'without-rrna-mapping')
    mkdir_p(symlink_dir)
    ribo_file = os.path.join(input_dir, 'RIBO_CCDS_lengths.bam')
    rna_file = os.path.join(input_dir, 'RNA_CCDS_lengths.bam')
    
    ribo_file_symlink = os.path.join(symlink_dir, 'RIBO_CCDS_lengths.alignments-only.bam')
    rna_file_symlink = os.path.join(symlink_dir, 'RNA_CCDS_lengths.alignments-only.bam')
    
    symlink_force(rna_file, rna_file_symlink)
    print(rna_file, rna_file_symlink)
    symlink_force(ribo_file, ribo_file_symlink)
    
    riboseq_data = os.path.join(input_dir, 'rpbp_results')
    config = """
    genome_name: {}
    genome_base_path: {}
    gtf: {}
    fasta: {}
    ribosomal_index: {}
    star_index: {}

    #adapter_file: /path/to/rp-bp/riboseq-adapters.fa

    riboseq_data: {}

    riboseq_samples:
        RIBO_CCDS_lengths: 
        RNA_CCDS_lengths:


    riboseq_biological_replicates:
        # these must still be the same names from riboseq_samples
        RIBO_CCDS_lengths:
            - RIBO_CCDS_lengths
        RNA_CCDS_lengths:
            - RNA_CCDS_lengths

    note: alignments-only

    orf_note: atg-only

    #models_base: /path/to/rp-bp/models
    max_uncalled: 1
    pre_trim_left: 0
    adapter_sequence: null
    quality_format: sanger


    sjdb_overhang: 50
    align_intron_minc: 20
    align_intron_maxc: 100000
    out_filter_intron_motifsc: RemoveNoncanonicalUnannotated
    out_filter_mismatch_n_maxc: 1
    out_filter_mismatch_n_over_l_maxc: 0.04
    out_filter_typec: BySJout
    out_sam_attributesc: [AS, NH, HI, nM, MD]

    # METAGENE PERIODICITY OPTIONS, OPTIONAL

    # the number of bases upstream of the translation initiation site to begin 
    # constructing the metagene profile. default: 50
    metagene_profile_start_upstream: 50 

    # the number of bases downstream of the translation initiation site to end 
    # the metagene profile. default: 20
    metagene_profile_start_downstream: 20

    # the number of bases upstream of the translation termination site to begin 
    # constructing the metagene profile. default: 50
    metagene_profile_end_upstream: 50

    # the number of bases downstream of the translation termination site to end 
    # the metagene profile. default: 20
    metagene_profile_end_downstream: 20

    # the position, relative to the translation initiation site, to begin 
    # calculating periodicity Bayes factors (inclusive)
    periodic_offset_start: -20

    # the position, relative to the translation initiation site, to stop 
    # calculating periodicity Bayes factors (inclusive)
    periodic_offset_end: 0

    # the length of the profile to use in the models. 

    # N.B. metagene_profile_length + periodic_offset_end must be consistent with 
    # the length of the extracted metagene profile
    metagene_profile_length: 21

    # the number of iterations to use for each chain in the MCMC sampling. The 
    # first half of the iterations are discarded as burn-in samples. All of 
    # the remaining samples are used to estimate the posterior distributions. That 
    # is, we do not use thinning. default: 500

    # N.B. Increasing the number of iterations will increase the running time of
    # the estimation roughly linearly.
    metagene_profile_iterations: 500

    # SHARED MARKOV CHAIN MONTE CARLO OPTIONS, OPTIONAL

    # These affect the MCMC both for estimating metagene profile periodicity and 
    # ORF translation Bayes factors.

    # the random seed for the MCMC sampling. default: 8675309
    seed: 8675309 

    # the number of chains to use in the MCMC sampling. default: 2

    # N.B. Increases in the number of chains should improve the quality of
    # estimation (especially reducing variance), but will increase the running
    # time proportionally. For example, changing the number of chains from 2 to
    # 4 will roughly double the running time of estimation.
    chains: 2

    # PERIODICITY AND OFFSET OPTIONS, OPTIONAL

    # FIXED LENGTHS AND OFFSETS
    # By default, the Rp-Bp pipelines use the periodicity estimation technique to
    # select periodic read lengths and P-site offsets. However, fixed lengths and
    # offsets can be specified with the following options.

    # if this variable is present in the config file with any value, then the 
    # estimated periodic read lengths and offsets will not be used. Instead, 
    # fixed values given by lengths and offsets (below) will be used.

    # uncomment this line to use the fixed lengths and offsets
    use_fixed_lengths: yes

    # a list of read lengths which will be used for creating the profiles if the 
    # use_fixed_lengths option is given. Presumably, these are lengths that have 
    # periodic metagene profiles.

    # This value is ignored if 'use_fixed_lengths' is not set.
    lengths: {}

    # the P-site offset to use for each read length specifed by --lengths if the 
    # use_fixed_lengths option is given. The number of offsets must match the 
    # number of lengths, and they are assumed to match. For example lengths of 
    # [26, 29] with offsets [9, 12] means only reads of lengths 26 bp and 29 bp 
    # will be used to create the profiles. The 26 bp reads will be shifted by 
    # 9 bp in the 5' direction, while reads of length 29 bp will be shifted by 12 bp.

    # This value is ignored if 'use_fixed_lengths' is not set.
    offsets: {}

    # if fixed lengths are not used, the minimum number of reads for a particular 
    # length in the filtered genome profile. Read lengths with fewer than this 
    # number of reads will not be used. default: 1000
    #min_metagene_profile_count: 1000

    # N.B. This value is set artificially low for the example to work!
    min_metagene_profile_count: 10

    # if fixed lengths are not used...
    # if max_metagene_profile_bayes_factor_var is not None, then this is taken as a 
    # hard threshold on the estimated Bayes factor mean. 

    # If min_metagene_bf_likelihood is given, then this is taken as 
    # the boundary value; that is, a profile is "periodic" if:
    #
    #       [P(bf > min_metagene_bf_mean)] > min_metagene_bf_likelihood
    #
    # If both max_metagene_bf_var and min_metagene_bf_likelihood are None, then this 
    # is taken as a hard threshold on the mean for selecting periodic read lengths.
    #
    # If both max_metagene_bf_var and min_metagene_bf_likelihood are given, then both 
    # filters will be applied and the result will be the intersection.
    min_metagene_bf_mean: 5

    # if given, then this is taken as a hard threshold on the estimated Bayes factor 
    # variance. default: None, i.e., this filter is not used. (null in yaml) 
    max_metagene_bf_var: null

    # if given, then this is taken a threshold on the likelihood of periodicity 
    # (see min_metagene_bf_mean description for more details). default: 0.5
    min_metagene_bf_likelihood: 0.5

    # SMOOTHING OPTIONS, OPTIONAL

    # the fraction of the profile to use for smoothing within LOWESS. default: 0.2
    smoothing_fraction: 0.2

    # the number of reweighting iterations to use within LOWESS. Please see the
    # statsmodels documentation for a detailed description of this parameter.
    # default: 0
    smoothing_reweighting_iterations: 0

    # if this value is greater than 0, then ORFs whose length (in nucleotides) 
    # is less than this value will not be smoothed, and neither the Bayes factor 
    # estimates nor the chi-square p-value will be calcualted. default: 0
    min_orf_length: 0

    # if this value is greater than 0, then ORFs whose length (in nucleotides) 
    # is greater than this value will not be smoothed, and neither the Bayes factor 
    # estimates nor the chi-square p-value will be calcualted. default: 0
    max_orf_length: 0

    # ORFs for which the number of in-frame reads is less than this value will not
    # be smoothed, and neither the Bayes factor estimates nor the chi-square
    # p-value will be calculated. default: 5
    min_signal: 5


    # BAYES FACTOR ESTIMATION OPTIONS, OPTIONAL

    # if this flag is in the config file with any value, then only the Rp-chi 
    # pipeline will be performed; namely, the translation models will not be fit 
    # to the data, and the posterior distributions will not be estimated.

    # uncomment this line to only run the Rp-chi pipeline
    #chi_square_only: yes

    # the number of iterations to use for each chain in the MCMC sampling. The first 
    # half of the iterations are discarded as burn-in samples. All of the remaining 
    # samples are used to estimate the posterior distributions. (That is, we do not 
    # use thinning.) default: 200

    # N.B. Increasing the number of iterations will increase the running time of
    # the estimation roughly linearly.
    translation_iterations: 200

    # SELECTION PREDICTED ORFS OPTIONS, OPTIONAL

    # the minimum value for the estimated Bayes factor mean to "predict" that an 
    # ORF is translated. This value is used in conjunction with both min_bf_mean
    # and min_bf_likelihood.
    #
    # If max_bf_var is a positive value, then this is taken as a hard threshold on 
    # the estimated Bayes factor mean. ORFs must meet both the min_bf_mean and
    # max_bf_var filters to be selected as "translated."
    #
    # If min_bf_likelihood is given, then this is taken as the boundary value; that
    # is, an ORF is selected as "translated" if:
    #
    #   [P(bf > min_bf_mean)] > min_bf_likelihood
    #
    # If both max_bf_var and min_bf_likelihood are None, then this is taken as a
    # hard threshold on the mean for selecting translated ORFs.
    #
    # If both max_bf_var and min_bf_likelihood are given, then both filters will 
    # be applied and the result will be the intersection.
    #
    # default: 5
    min_bf_mean: 5

    # the maximum value value for the estimated Bayes factor variance to "predict" 
    # that an ORF is translated. ORFs must meet both the min_bf_mean and max_bf_var 
    # filters to be predicted. See the description for min_bf_mean for more 
    # details. default: null (i.e., this filter is not used by default)
    max_bf_var: null

    # the minimum probability of the BF exceeding min_bf_mean to select an ORF as
    # translated. See the description for min_bf_mean for more details. default: 0.5
    min_bf_likelihood: 0.5

    # for the chi-square test, this value is first Bonferroni corrected 
    # based on the number of ORFs which pass the smoothing filters. It is 
    # then used as the significance threshold to select translated ORFs. default: 0.01
    chisq_significance_level: 0.01
    """.format(genome_name, genome_base_path, gtf,
               genome_fasta, ribosomal_index, star_index,
               riboseq_data, lengths, offsets)
    config = config.replace("'","")
    with open('/home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/rpbp/config_{}_mouse.yaml'.format(srp), 'w') as fh:
        fh.write(dedent(config))
    to_write = 'run-all-rpbp-instances config_{}_mouse.yaml --overwrite --num-cpus 16 --logging-level INFO --log-file log.alignments-only_{}_mouse.txt --keep-intermediate-files'.format(srp, srp)
    with open('/home/cmb-panasas2/skchoudh/github_projects/ribocop-results/tools/rpbp/run_rpbp_{}_mouse.sh'.format(srp), 'w') as fh:
        fh.write(to_write)

/home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/SRP003554_mouse/RNA_CCDS_lengths.bam /home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/SRP003554_mouse/rpbp_results/without-rrna-mapping/RNA_CCDS_lengths.alignments-only.bam
/home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/SRP062407_mouse/RNA_CCDS_lengths.bam /home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/SRP062407_mouse/rpbp_results/without-rrna-mapping/RNA_CCDS_lengths.alignments-only.bam
/home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/SRP078005_mouse/RNA_CCDS_lengths.bam /home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/SRP078005_mouse/rpbp_results/without-rrna-mapping/RNA_CCDS_lengths.alignments-only.bam
/home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/SRP091889_mouse/RNA_CCDS_lengths.bam /home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/SRP091889_mouse/rpbp_results/without-rrna-mapping/RNA_CCDS_lengths.alignments-only.bam
/home/cmb-panasas2/skchoudh/benchmark/gene_level/mm10/SRP115915_mouse/RN